In [1]:
import os
from PIL import Image
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split
import tensorflow as tf
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import torch
from PIL import Image
from transformers import AutoModelForImageClassification
import tensorflow as tf

In [2]:
# Set image dimensions
IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 3

In [3]:
def conv_block(inputs, num_filters):
    x = tf.keras.layers.Conv2D(num_filters, (3, 3), activation="relu", 
                               kernel_initializer="he_normal", padding="same")(inputs)
    x = tf.keras.layers.Dropout(0.1)(x)
    x = tf.keras.layers.Conv2D(num_filters, (3, 3), activation="relu", 
                               kernel_initializer="he_normal", padding="same")(x)
    return x

In [4]:
def upsample_block(inputs, conv_prev, num_filters):
    up = tf.keras.layers.Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding="same")(inputs)
    concat = tf.keras.layers.concatenate([up, conv_prev])
    conv = conv_block(concat, num_filters)
    return conv

In [5]:
# Input layer
inputs = tf.keras.layers.Input((IMG_WIDTH, IMG_HEIGHT, IMG_CHANNELS))
s = tf.keras.layers.Lambda(lambda x: x / 255.0)(inputs)

In [6]:
# Contracting path
c1 = conv_block(s, 16)
p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

c2 = conv_block(p1, 32)
p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)

c3 = conv_block(p2, 64)
p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)

c4 = conv_block(p3, 128)
p4 = tf.keras.layers.MaxPooling2D((2, 2))(c4)

c5 = conv_block(p4, 256)
c6 = upsample_block(c5, c4, 128)
c7 = upsample_block(c6, c3, 64)
c8 = upsample_block(c7, c2, 32)
c9 = upsample_block(c8, c1, 16)

In [7]:
# Output layer (single channel for binary segmentation, sigmoid activation for probability output)
outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)

In [8]:
# Load the saved model
loaded_model = tf.keras.models.load_model('/kaggle/input/u-net/other/default/1/model.h5')

# Verify by inspecting the model summary
loaded_model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lambda (Lambda)     │ (None, 128, 128,  │          0 │ input_layer[0][0] │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 128, 128,  │        448 │ lambda[0][0]      │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 128, 128,  │          0 │ conv2d[0][0]      │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 128, 128,  │      2,320 │ dropout[0][0]     │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 64, 64,    │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 64, 64,    │      4,640 │ max_pooling2d[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 64, 64,    │          0 │ conv2d_2[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 64, 64,    │      9,248 │ dropout_1[0][0]   │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 32, 32,    │          0 │ conv2d_3[0][0]    │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 32, 32,    │     18,496 │ max_pooling2d_1[… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 32, 32,    │          0 │ conv2d_4[0][0]    │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 32, 32,    │     36,928 │ dropout_2[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 16, 16,    │          0 │ conv2d_5[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 16, 16,    │     73,856 │ max_pooling2d_2[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 16, 16,    │          0 │ conv2d_6[0][0]    │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_7 (Conv2D)   │ (None, 16, 16,    │    147,584 │ dropout_3[0][0] 

 Total params: 1,941,107 (7.40 MB)

 Trainable params: 1,941,105 (7.40 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [9]:
# Function to load images and masks, resizing them to the target dimensions
def load_images_from_dir(image_dir, mask_dir):
    images = []
    masks = []
    
    image_subfolders = os.listdir(image_dir)
    
    for subfolder in image_subfolders:
        subfolder_image_path = os.path.join(image_dir, subfolder)
        subfolder_mask_path = os.path.join(mask_dir, subfolder)

        if os.path.isdir(subfolder_image_path):
            image_files = os.listdir(subfolder_image_path)

            for image_file in image_files:
                image_path = os.path.join(subfolder_image_path, image_file)
                mask_file = find_mask_for_image(image_file, subfolder_mask_path)
                
                if mask_file:
                    mask_path = os.path.join(subfolder_mask_path, mask_file)
                    
                    try:
                        # Load and resize images and masks
                        image = Image.open(image_path).resize((IMG_WIDTH, IMG_HEIGHT))
                        mask = Image.open(mask_path).resize((IMG_WIDTH, IMG_HEIGHT))
                        
                        # Convert to numpy arrays
                        image = np.array(image)
                        mask = np.array(mask)
                        
                        # Append to lists
                        images.append(image)
                        masks.append(mask)
                    except Exception as e:
                        print(f"Error loading image or mask {image_file}: {e}")
                else:
                    print(f"No matching mask found for image {image_file}. Skipping.")
    
    return np.array(images), np.array(masks)

def find_mask_for_image(image_file, mask_dir):
    image_name_without_extension = os.path.splitext(image_file)[0]
    mask_files = os.listdir(mask_dir)
    for mask_file in mask_files:
        if image_name_without_extension in mask_file:
            return mask_file
    return None

val_image_dir = "/kaggle/input/cv25-project-dataset/Project Data/Project Data/Val/images"
val_mask_dir = "/kaggle/input/cv25-project-dataset/Segmented Data/Segmented Data/Val"

val_images, val_masks = load_images_from_dir(val_image_dir, val_mask_dir)
val_images = val_images / 255.0
val_masks = (val_masks > 0).astype(np.float32)

# Evaluate
loss, accuracy = loaded_model.evaluate(val_images, val_masks)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")

I0000 00:00:1734542552.804589      89 service.cc:145] XLA service 0x7bd134005360 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1734542552.804675      89 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0


10/11 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9562 - loss: 0.1253

I0000 00:00:1734542555.666786      89 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 141ms/step - accuracy: 0.9564 - loss: 0.1249
Validation Loss: 0.12240565568208694
Validation Accuracy: 0.9574074745178223
